### Need a Harry Potter Chat
- I ask ask any query related to harry potter
-  Out of box question :  respond no
-  RAG + Agent
-  Prerequsite:
   -  Harry Potter - full text data 
   -  RAG - Database : `PineCone` 
   -  Embedding Model :Opneai
   -  Tracing, Evaluation, Deployment
   -   Tracing, to mponiter, the step by step invocatioon and llm call of an agent
   -   Evaluation :  How well you agent is perfoming
   -   Deployment : Everyone can use

## Things need to check
 - Embeddding model context length -- how it works
 - Loading / Cleaning
 - Clean - Cutting
 -  Evaluate RAG
 -      Perfomce = 80

In [113]:
### Data loading
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

loader = PyMuPDFLoader(file_path="./harry.pdf")



In [121]:
normal_load = loader.load()


In [122]:
len(normal_load)

3623

In [123]:
## Splitting the data
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size =1000,
    chunk_overlap=200
)

## gpt-5-mini = 400k , 40k(10%)
## pdf : 800 - 1200


In [124]:
docs = splitter.split_documents(normal_load)

In [130]:
len(docs)

8887

In [ ]:
## Embedding
from langchain_openai.embeddings import OpenAIEmbeddings
import tiktoken

# Price: About $0.13 per 1M tokens
text = "Harry potter is a cursed child"

encoding = tiktoken.encoding_for_model("text-embedding-3-large")

len(encoding.encode(text))
len(encoding.encode(normal_load[5].page_content))
sum([len(encoding.encode(page.page_content)) for page in normal_load])

7

In [127]:
## Embedding
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
import os
from dotenv import load_dotenv
load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

vector_db  =  Chroma(
    collection_name="harry_collection",
    embedding_function=embedding,
    chroma_cloud_api_key=os.getenv("CHROMA_API_KEY"),
    tenant=os.getenv("CHROMA_TENANT"),
    database=os.getenv("CHROMA_DATABASE"),
)


In [128]:
for i in range(0,len(docs),300):
    vector_db.add_documents(documents=docs[i:i+300])


In [129]:
len(docs)

8887

In [ ]:

prompt = "In the harry potter series, who is refferred as Half Blood Prince?"

out = vector_db.similarity_search_with_relevance_scores(prompt, k=20)


ImportError: cannot import name 'ContextualCompressionRetriever' from 'langchain_core.retrievers' (/Users/sachin/Desktop/Production_Agents/.venv/lib/python3.12/site-packages/langchain_core/retrievers.py)

In [201]:
from rich.markdown import Markdown


print(out[1][0].page_content)


my particular brand of reasoned argument is making much headway against
Greyback’s insistence that we werewolves deserve blood, that we ought to
revenge ourselves on normal people.”
“But you are normal!” said Harry fiercely. “You’ve just got a — a problem
—”
Lupin burst out laughing. “Sometimes you remind me a lot of James. He
called it my ‘furry little problem’ in company. Many people were under the
impression that I owned a badly behaved rabbit.”
He accepted a glass of eggnog from Mr. Weasley with a word of thanks,
looking slightly more cheerful. Harry, meanwhile, felt a rush of excitement:
This last mention of his father had reminded him that there was something he
had been looking forward to asking Lupin.
“Have you ever heard of someone called the Half-Blood Prince?”
“The Half-Blood what?”
“Prince,” said Harry, watching him closely for signs of recognition.
“There are no Wizarding princes,” said Lupin, now smiling. “Is this a title


In [202]:
for score in out:
    print(score[-1])


0.5651803826798749
0.4054069891756842
0.38145982808130807
0.3799342169166269
0.3464362352394422
0.3353686281845816
0.3300283032146195
0.30439876538217314
0.30007639133440733
0.29955010589930575
0.28539173510387883
0.279558174869768
0.2777522948612955
0.2658031094980584
0.26483345396901736
0.2609636706876177
0.2574429860240899
0.25082835563948036
0.25014741180919775
0.24106285743790357
